<a href="https://colab.research.google.com/github/fjgr/IA_BigData/blob/main/M2D/Tarea-7/TAREA_7_WHISPER_(Hugging_Face).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad con WHISPER (Hugging Face)

## 📌 1. Instalación de dependencias
Esta celda instala todas las bibliotecas necesarias para el proyecto.

Se incluyen:
 - **transformers**: para procesamiento de lenguaje natural
 - **torch**: para modelos de deep learning
 - **ffmpeg**: para conversión de audio
 - **openai-whisper**: modelo de transcripción
 - **pydub**: para análisis de audio
 - **tqdm**: para mostrar progreso

In [1]:
!pip install transformers torch ffmpeg openai-whisper pydub tqdm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

## 📌 2. Importación de librerías necesarias
En esta celda, importamos las bibliotecas que usaremos para:
- Cargar y analizar archivos de audio.
- Convertir archivos si es necesario.
- Ejecutar el modelo de transcripción Whisper.
- Mostrar el progreso con barras de carga.


In [2]:
import whisper
import torchaudio
import subprocess
from IPython.display import Audio
from google.colab import files
from pydub.utils import mediainfo
from pathlib import Path
from tqdm import tqdm

## 📌 3. Función para verificar el formato del audio
Antes de procesar el archivo, es importante verificar:

✅ Si el audio tiene una frecuencia de **16 kHz**.  
✅ Si el audio está en **mono (1 canal)**.  

Si el audio no cumple con estos requisitos, lo convertiremos.


In [3]:
def verificar_audio(audio_file):
    """
    Analiza el archivo de audio y verifica si es compatible con Whisper.

    Parámetros:
    - audio_file (str): Ruta del archivo de audio a analizar.

    Retorna:
    - True si el archivo ya está en formato correcto (16kHz, mono).
    - False si el archivo necesita conversión.
    """
    try:
        info = mediainfo(audio_file)
        sample_rate = int(info['sample_rate'])  # Frecuencia de muestreo
        channels = int(info['channels'])  # Número de canales
        format = info['format_name']  # Formato del archivo

        print(f"🎵 Formato: {format}, 🎚️ Frecuencia: {sample_rate} Hz, 🔊 Canales: {channels}")

        if sample_rate != 16000 or channels != 1:
            print("⚠️ El archivo necesita conversión a 16kHz mono.")
            return False
        return True
    except Exception as e:
        print(f"⚠️ Error al analizar el archivo: {e}")
        return False

## 📌 4. Función para convertir audio con FFmpeg
Si el audio **no está en formato compatible**, usamos **FFmpeg** para convertirlo.
- Se cambia a **16 kHz** (frecuencia de muestreo).
- Se convierte a **mono** (1 solo canal de audio).


In [4]:
def convertir_audio(input_file, output_file):
    """
    Convierte el archivo de audio a 16kHz mono usando FFmpeg.

    Parámetros:
    - input_file (str): Ruta del archivo de entrada.
    - output_file (str): Ruta del archivo de salida convertido.

    Retorna:
    - True si la conversión fue exitosa.
    - False si hubo un error.
    """
    cmd = [
        "ffmpeg", "-y", "-i", input_file,
        "-ac", "1", "-ar", "16000", output_file
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print("⚠️ Error en la conversión de audio:", result.stderr)
        return False
    return True

## 📌 5. Subir archivo de audio
En esta celda, subimos un archivo de audio (.mp3, .wav, etc.).


In [5]:
uploaded = files.upload()  # Permite seleccionar un archivo desde el PC
audio_file = list(uploaded.keys())[0]  # Guarda el nombre del archivo subido
audio_path = Path(audio_file)  # Convierte la ruta a un objeto Path
converted_audio = audio_path.with_name("audio_converted.wav")  # Ruta del archivo convertido

Saving Nada Valgo Sin Tu Amor.wav to Nada Valgo Sin Tu Amor.wav


## 📌 6. Verificar y convertir el archivo (si es necesario)
Antes de usar Whisper, verificamos si el archivo necesita conversión.

Si no cumple con los requisitos, **lo convertimos a 16 kHz mono**.


In [6]:
if not verificar_audio(str(audio_path)):
    print("⏳ Convirtiendo archivo a 16kHz mono...")
    if convertir_audio(str(audio_path), str(converted_audio)):
        print("✅ Conversión completada.")
        audio_path = converted_audio
    else:
        print("❌ No se pudo convertir el archivo.")
else:
    print("✅ El archivo ya está en formato correcto.")

🎵 Formato: wav, 🎚️ Frecuencia: 44100 Hz, 🔊 Canales: 2
⚠️ El archivo necesita conversión a 16kHz mono.
⏳ Convirtiendo archivo a 16kHz mono...
✅ Conversión completada.


## 📌 7. Reproducir el audio convertido
Después de la conversión, aseguramos que el archivo convertido **existe** antes de reproducirlo.


In [7]:
if converted_audio.exists():
    Audio(str(converted_audio))  # Reproducir el archivo en Google Colab
else:
    print("⚠️ Error: El archivo de audio convertido no existe.")

## 📌 8. Cargar el modelo Whisper
Ahora cargamos el modelo **"large"** de Whisper.  
**Nota:** Después de probar otros modelos como el "small" y el "medium", el **"large"** es el que mejor realiza la transcripción.  


In [8]:
model = whisper.load_model("large")  # Cargar modelo de Whisper

100%|█████████████████████████████████████| 2.88G/2.88G [01:52<00:00, 27.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

## 📌 9. Transcribir el audio
Ejecutamos Whisper para transcribir el archivo **con una barra de progreso**.


In [9]:
print("⏳ Transcribiendo audio...")
with tqdm(total=100) as pbar:
    result = model.transcribe(str(audio_path))
    pbar.update(100)

# Mostrar la transcripción final
print("\n🎤 **Transcripción:**")
print(result["text"])

⏳ Transcribiendo audio...


100%|██████████| 100/100 [00:52<00:00,  1.92it/s]


🎤 **Transcripción:**
 Cuando el tiempo pasa y nos hacemos viejos, nos empieza a parecer que pesan más los daños que los mismos años al final. Por eso yo quiero que mis años pasen junto a ti mi amor eterno, junto a mi familia, junto a mis amigos y mi voz. Porque nada valgo, porque nada tengo, si no tengo lo mejor, tu amor y compañía en mi corazón. Y es que vale más un año tardío que un siglo vacío amor, y es que vale más tener bien llenito el corazón. Por eso yo quiero que en mi mente siempre tu cariño. Que tu cariño esté bien fuerte, aunque estemos lejos o aunque estemos cerca del final. Porque nada valgo, porque nada tengo, si no tengo lo mejor, tu amor y compañía en mi corazón. Ven amor, me siento débil cuando estoy sin ti, me hago fuerte cuando estás aquí. Sin ti yo ya no sé qué es. Mi vida es un túnel sin tu luz. Quiero pasar más tiempo junto a ti, recuperar las noches que perdí. Ven ser el miedo inmenso de morir y ser eterno junto a ti. Porque nada valgo, porque nada tengo, si no

## 📌 10. Evaluar la transcripción
Finalmente, podemos comparar la transcripción obtenida con la letra original de la canción.


In [32]:
import difflib
import re
import nltk
from difflib import SequenceMatcher

# Descargar el tokenizer de NLTK si es necesario
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Texto original de la canción
texto_original = """Cuando el tiempo pasa y nos hacemos viejos
Nos empieza a parecer
Que pesan más los daños que los mismos años, al final
Por eso yo quiero que mis años pasen
Junto a ti, mi amor eterno
Junto a mi familia, junto a mis amigos y mi voz
Porque nada valgo, porque nada tengo
Si no tengo lo mejor
Tu amor y compañía en mi corazón
Y es que vale más un año tardío que un siglo vacío, amor
Y es que vale más tener bien llenito el corazón
Por eso yo quiero que, en mi mente siempre tu cariño esté bien fuerte
Aunque estemos lejos o, aunque estemos cerca del final
Porque nada valgo, porque nada tengo
Si no tengo lo mejor
Tu amor y compañía en mi corazón, ¡ven amor!
Me siento débil cuando estoy sin ti
Y me hago fuerte cuando estás aquí
Sin ti yo ya no sé qué es vivir
Mi vida es un túnel sin tu luz
Quiero pasar más tiempo junto a ti
Recuperar las noches que perdí
Vencer el miedo inmenso de morir
Y ser eterno junto a ti
Porque nada valgo, porque nada tengo
Si no tengo lo mejor
Tu amor y compañía en mi corazón
Por eso yo quiero que, en mi mente siempre tu cariño esté bien fuerte
Aunque estemos lejos o aunque estemos cerca del final
Porque nada valgo, porque nada tengo
Si no tengo lo mejor
Tu amor y compañía en mi corazón, ¡ven amor!
Me siento débil cuando estoy sin ti
Y me hago fuerte cuando estás aquí
Sin ti yo ya no sé qué es vivir
Mi vida es un túnel sin tu luz
Quiero pasar más tiempo junto a ti
Recuperar las noches que perdí
Vencer el miedo inmenso de morir
Y ser eterno junto a ti
Porque nada valgo, porque nada tengo
Si no tengo lo mejor
Tu amor y compañía en mi corazón"""

# Transcripción obtenida de Whisper
texto_transcrito = result["text"]

# Función para limpiar el texto (quitar puntuación y convertir a minúsculas)
def limpiar_texto(texto):
    texto = texto.lower().strip()  # Convertir a minúsculas y quitar espacios extra
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar puntuación
    return texto

# Limpiar la transcripción y el texto original
texto_original_limpio = limpiar_texto(texto_original).split("\n")
texto_transcrito_limpio = limpiar_texto(texto_transcrito)

# Función para encontrar la mejor coincidencia dentro de la transcripción
def mejor_coincidencia(linea, texto_completo):
    palabras = texto_completo.split()  # Dividir transcripción en palabras
    max_similitud = 0
    mejor_fragmento = ""

    for i in range(len(palabras)):
        for j in range(i + 3, min(i + 10, len(palabras))):  # Buscar fragmentos de 3 a 10 palabras
            fragmento = " ".join(palabras[i:j])
            similitud = SequenceMatcher(None, linea, fragmento).ratio()
            if similitud > max_similitud:
                max_similitud = similitud
                mejor_fragmento = fragmento

    return mejor_fragmento, max_similitud

# Comparar cada línea del texto original con la transcripción
print("🔍 Comparación entre la letra original y la transcripción:\n")

# Inicializar total_similitud y total_lineas antes del bucle
total_similitud = 0
total_lineas = 0

for linea in texto_original_limpio:
    coincidencia, similitud = mejor_coincidencia(linea, texto_transcrito_limpio)
    total_similitud += similitud  # Acumular el porcentaje de similitud
    total_lineas += 1  # Contar la línea
    print(f"🔸 Original: {linea}")
    print(f"🔹 Transcrito: {coincidencia}")
    print(f"📊 Similitud: {similitud:.2%}\n")

# Calcular el porcentaje total promedio
porcentaje_total = total_similitud / total_lineas if total_lineas > 0 else 0
print(f"🎯 **Similitud total de la canción:** {porcentaje_total:.2f}%")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


🔍 Comparación entre la letra original y la transcripción:

🔸 Original: cuando el tiempo pasa y nos hacemos viejos
🔹 Transcrito: cuando el tiempo pasa y nos hacemos viejos
📊 Similitud: 100.00%

🔸 Original: nos empieza a parecer
🔹 Transcrito: nos empieza a parecer
📊 Similitud: 100.00%

🔸 Original: que pesan más los daños que los mismos años al final
🔹 Transcrito: que pesan más los daños que los mismos años
📊 Similitud: 90.53%

🔸 Original: por eso yo quiero que mis años pasen
🔹 Transcrito: por eso yo quiero que mis años pasen
📊 Similitud: 100.00%

🔸 Original: junto a ti mi amor eterno
🔹 Transcrito: junto a ti mi amor eterno
📊 Similitud: 100.00%

🔸 Original: junto a mi familia junto a mis amigos y mi voz
🔹 Transcrito: junto a mi familia junto a mis amigos y
📊 Similitud: 91.76%

🔸 Original: porque nada valgo porque nada tengo
🔹 Transcrito: porque nada valgo porque nada tengo
📊 Similitud: 100.00%

🔸 Original: si no tengo lo mejor
🔹 Transcrito: si no tengo lo mejor
📊 Similitud: 100.00%

🔸 Ori